# Introduction

SQL is based on a client-server architecture.


Python can connect with every SQL databases (Oracle MySQL, Microsoft, etc...). SQLite is a little bit particular : it is a little file, who can play the server role, without username and password. It will be helpful for this quest. But keep in mind that you could connect your Python script with other databases, following the same steps.

In [ ]:
# Here we just import the SQLite file. This part is specific for SQLite.
import requests
r = requests.get('https://raw.githubusercontent.com/murpi/wilddata/master/quests/toys_and_models.sqlite')
open('toys_and_models.sqlite', 'wb').write(r.content)

307200

In [ ]:
# This part is common : we have to initiate a connection between Python and the SQL Database
import sqlite3

# This is the "connector". For secured database, it will be here that  you indicate your username and password.
conn = sqlite3.connect('toys_and_models.sqlite')

# This is the "cursor". A cursor is an object that can execute a query and retrieve the content of the result.
cursor = conn.cursor()


In [ ]:
# Pandas can import the result of a query
import pandas as pd
pd.DataFrame(cursor.execute("select * from productlines").fetchall())

,0,1,2,3
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,None,None
1,Motorcycles,Our motorcycles are state of the art replicas ...,None,None
2,Planes,"Unique, diecast airplane and helicopter replic...",None,None
3,Ships,The perfect holiday or anniversary gift for ex...,None,None
4,Trains,Model trains are a rewarding hobby for enthusi...,None,None
5,Trucks and Buses,The Truck and Bus models are realistic replica...,None,None
6,Vintage Cars,Our Vintage Car models realistically portray a...,None,None


![Texte alternatif…](https://mysqltutorial.org/wp-content/uploads/2009/12/MySQL-Sample-Database-Schema.png)

## SALES
The number of products sold by category and by month, with comparison and rat of change compared to the same month of the previous year.

In [ ]:
query1 = '''
  WITH temp AS (
    SELECT
      productLine,
      CAST(strftime('%m', orderDate) as INTEGER) as month,
      CAST(strftime('%Y', orderDate) as INTEGER) as year,
      COUNT(*) as sales
    FROM
      orderdetails
      NATURAL JOIN products
      NATURAL JOIN orders
    GROUP BY productLine, year, month
  )
  SELECT
    *,
    CASE
      WHEN month = 1 THEN 'January'
      WHEN month = 2 THEN 'February'
      WHEN month = 3 THEN 'March'
      WHEN month = 4 THEN 'April'
      WHEN month = 5 THEN 'May'
      WHEN month = 6 THEN 'June'
      WHEN month = 7 THEN 'July'
      WHEN month = 8 THEN 'August'
      WHEN month = 9 THEN 'September'
      WHEN month = 10 THEN 'October'
      WHEN month = 11 THEN 'November'
      WHEN month = 12 THEN 'December'
      ELSE 'Invalid Month'
    END AS month_name,
    LAG(sales, 12) OVER (PARTITION BY productLine ORDER BY year, month) AS prev_year_sales,
    CAST(sales AS REAL) / CAST(LAG(sales, 12) OVER (PARTITION BY productLine ORDER BY year, month) AS REAL) * 100 AS change_rate
    FROM temp
    ORDER BY change_rate DESC, year ASC, month ASC, productLine;

'''

df1 = pd.read_sql(query1, conn)
# df1.to_csv('sales.csv', index=False)
df1

,productLine,month,year,sales,month_name,prev_year_sales,change_rate
0,Motorcycles,10,2019,13,October,1.0,1300.0
1,Classic Cars,2,2019,33,February,3.0,1100.0
2,Trucks and Buses,2,2020,8,February,1.0,800.0
3,Motorcycles,7,2019,7,July,1.0,700.0
4,Classic Cars,8,2019,55,August,10.0,550.0
...,...,...,...,...,...,...,...
156,Trains,3,2019,3,March,NaN,NaN
157,Planes,4,2019,9,April,NaN,NaN
158,Planes,5,2019,12,May,NaN,NaN
159,Ships,5,2019,9,May,NaN,NaN


## FINANCES

Sales by countries for last 2 months

In [ ]:
query2 = '''
  SELECT TRIM(country),
    SUM(priceEach * quantityOrdered) AS country_sales_2_month,
    AVG(SUM(priceEach * quantityOrdered)) OVER () as average_2_months_sales,
    CASE
      WHEN SUM(priceEach * quantityOrdered) > AVG(SUM(priceEach * quantityOrdered)) OVER () THEN 'More then average sales'
      WHEN SUM(priceEach * quantityOrdered) = AVG(SUM(priceEach * quantityOrdered)) OVER () THEN 'Equals average sales'
      WHEN SUM(priceEach * quantityOrdered) < AVG(SUM(priceEach * quantityOrdered)) OVER () THEN 'Less then average sales'
      ELSE 'Non calculable'
    END AS average_sales_flag
  FROM orderdetails NATURAL JOIN products NATURAL JOIN orders JOIN customers
  WHERE orderDate BETWEEN
    (SELECT strftime('%Y-%m-%d', MAX(orderDate), 'start of month', '-1 month') FROM orders)
    AND
    (SELECT MAX(orderDate) FROM orders)
    GROUP BY TRIM(country)
  ORDER BY TRIM(country);
'''


df2 = pd.read_sql(query2, conn)
# df2.to_csv('finances.csv', index=False)
df2

,TRIM(country),country_sales_2_month,average_2_months_sales,average_sales_flag
0,Australia,2830079.15,2.557553e+06,More then average sales
1,Austria,1132031.66,2.557553e+06,Less then average sales
2,Belgium,1132031.66,2.557553e+06,Less then average sales
3,Canada,1698047.49,2.557553e+06,Less then average sales
4,Denmark,1132031.66,2.557553e+06,Less then average sales
5,Finland,1698047.49,2.557553e+06,Less then average sales
6,France,6792189.96,2.557553e+06,More then average sales
7,Germany,7358205.79,2.557553e+06,More then average sales
8,Hong Kong,566015.83,2.557553e+06,Less then average sales
9,Ireland,1132031.66,2.557553e+06,Less then average sales


Unfinished orders

In [ ]:
query3 = '''
  ????
'''

df3 = pd.read_sql(query3, conn)
df3

## Logistics

Stock of 5 most ordered

In [ ]:
query4 = '''
SELECT productCode, productName, quantityInStock,
        RANK() OVER(ORDER BY SUM(quantityOrdered) DESC) as rank
        FROM orderdetails o
        NATURAL LEFT JOIN products pr
        GROUP BY productCode
        ORDER BY SUM(quantityOrdered) DESC
        LIMIT 5;
'''

df4 = pd.read_sql(query4, conn)
# df4.to_csv('logistics.csv', index=False)
df4

,productCode,productName,quantityInStock,rank
0,S18_3232,1992 Ferrari 360 Spider red,8347,1
1,S18_1342,1937 Lincoln Berline,8693,2
2,S12_1108,2001 Ferrari Enzo,3619,3
3,S18_2949,1913 Ford Model T Speedster,4189,4
4,S18_4600,1940s Ford truck,3128,5


## HUMAN RESOURCES
2 top sellers for each months

In [ ]:
query5 = '''
WITH temp AS (
    SELECT
        employeeNumber,
        CAST(strftime('%m', paymentDate) as INTEGER) as month,
        CAST(strftime('%Y', paymentDate) as INTEGER) as year,
        SUM(amount) AS monthly_sales,
        RANK() OVER (
            PARTITION BY
                CAST(strftime('%m', paymentDate) as INTEGER),
                CAST(strftime('%Y', paymentDate) as INTEGER)
            ORDER BY SUM(amount) DESC
        ) as sales_rank,
        FIRST_VALUE(employeeNumber) OVER (PARTITION BY CAST(strftime('%Y', paymentDate) as INTEGER) ORDER BY SUM(amount) DESC) as best_salesman_this_year
    FROM employees e
    JOIN customers c ON c.salesRepEmployeeNumber = e.employeeNumber
    NATURAL JOIN payments p
    GROUP BY employeeNumber, month, year
)
SELECT month, year, employeeNumber, sales_rank, monthly_sales, best_salesman_this_year
FROM temp
WHERE sales_rank in (1, 2)
ORDER BY year, month;
'''

df5 = pd.read_sql(query5, conn)
df5.to_csv('hr.csv', index=False)
df5

,month,year,employeeNumber,sales_rank,monthly_sales,best_salesman_this_year
0,1,2018,1504,1,10549.01,1165
1,1,2018,1216,2,10223.83,1165
2,2,2018,1401,1,53959.21,1165
3,2,2018,1504,2,50218.95,1165
4,3,2018,1401,1,52151.81,1165
5,3,2018,1621,2,51001.22,1165
6,4,2018,1621,1,44380.15,1165
7,4,2018,1337,2,33383.14,1165
8,5,2018,1337,1,67525.13,1165
9,5,2018,1611,2,53429.11,1165



![Texte alternatif…](https://mysqltutorial.org/wp-content/uploads/2009/12/MySQL-Sample-Database-Schema.png)

## Tiresome experimets

In [ ]:
query69 = '''
WITH order_prices AS (
  SELECT o.orderNumber, SUM(priceEach * quantityOrdered) as order_sale
  FROM orderdetails d NATURAL JOIN orders o
  GROUP BY o.orderNumber
)
SELECT o.orderNumber, amount
FROM order_prices op
NATURAL JOIN orders o
LEFT JOIN payments p ON p.paymentDate = o.orderDate AND p.customerNumber = o.customerNumber
'''

help_query = '''
  SELECT status, COUNT(*) FROM orders GROUP BY status;
'''

q1 = '''
  SELECT * FROM payments WHERE paymentDate = '2018-01-16';
'''

q2 = '''
  SELECT COUNT(*) FROM payments;
'''




df69 = pd.read_sql(q1, conn)
df69

,customerNumber,checkNumber,paymentDate,amount
0,363,IS232033,2018-01-16,10223.83


In [ ]:
q6 = '''
  WITH payment_summary AS (
    SELECT paymentDate,
      CAST(strftime('%m', paymentDate) as INTEGER) as month,
      CAST(strftime('%Y', paymentDate) as INTEGER) as year,
      amount
    FROM payments
  )
  SELECT
      year,
      month,
      CASE
        WHEN month = 1 THEN 'January'
        WHEN month = 2 THEN 'February'
        WHEN month = 3 THEN 'March'
        WHEN month = 4 THEN 'April'
        WHEN month = 5 THEN 'May'
        WHEN month = 6 THEN 'June'
        WHEN month = 7 THEN 'July'
        WHEN month = 8 THEN 'August'
        WHEN month = 9 THEN 'September'
        WHEN month = 10 THEN 'October'
        WHEN month = 11 THEN 'November'
        WHEN month = 12 THEN 'December'
        ELSE 'Invalid Month'
      END AS month_name,
      SUM(amount) as monthly_sales,
      SUM(SUM(amount)) OVER (ORDER BY paymentDate) AS run_total,
      AVG(SUM(amount)) OVER (ORDER BY paymentDate) AS run_average,
      SUM(SUM(amount)) OVER (PARTITION BY year ORDER BY paymentDate) AS run_total_annualy,
      AVG(SUM(amount)) OVER (PARTITION BY year ORDER BY paymentDate) AS run_average_annualy

  FROM payment_summary
  GROUP BY year, month
  ORDER BY year, month;
'''


df6 = pd.read_sql(q6, conn)
# df6.to_csv('running_total_sales.csv', index=False)
df6

,year,month,month_name,monthly_sales,run_total,run_average,run_total_annualy,run_average_annualy
0,2018,1,January,26267.62,26267.62,26267.620000,26267.62,26267.620000
1,2018,2,February,144384.36,170651.98,85325.990000,170651.98,85325.990000
2,2018,3,March,199704.48,370356.46,123452.153333,370356.46,123452.153333
3,2018,4,April,136313.92,506670.38,126667.595000,506670.38,126667.595000
4,2018,5,May,159881.97,666552.35,133310.470000,666552.35,133310.470000
5,2018,6,June,180218.98,846771.33,141128.555000,846771.33,141128.555000
6,2018,7,July,158247.00,1005018.33,143574.047143,1005018.33,143574.047143
7,2018,8,August,246204.86,1251223.19,156402.898750,1251223.19,156402.898750
8,2018,9,September,161206.23,1412429.42,156936.602222,1412429.42,156936.602222
9,2018,10,October,316857.96,1729287.38,172928.738000,1729287.38,172928.738000
